In [ ]:
import os
import re,string
import nltk
import re
import spacy
import pandas as pd
import numpy as np
import networkx as nx
import bs4
nlp = spacy.load('en_core_web_sm')
from tqdm import tqdm
from collections import defaultdict
import time
import matplotlib.pyplot as plt


nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import MDS
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold

from gensim.models import Word2Vec,LdaMulticore, TfidfModel
from gensim import corpora
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import KeyedVectors

from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout

!pip install nlpia
from nlpia.data.loaders import get_data
!pip install python-docx
from docx import Document
import time
from itertools import combinations
from itertools import permutations
from itertools import chain
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import sparse_categorical_crossentropy
from keras.utils import plot_model
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Embedding, SimpleRNN
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 32.0MB 134kB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 716kB 50.1MB/s 
  Created wheel for pypandoc: filename=pypandoc-1.5-cp36-none-any.whl size=17038 sha256=7294d8209282786b6c9dde3c9a646040dc220440fec368120b2dde23f6e55a6a
  Stored in directory: /root/.cache/pip/wheels/bb/7d/d6/2f9af55e800d37e42e546106bcbd36a86e24e725e303d17e04
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144793 sha256=5b96b5f09a3b5eed84b7902a7aa0438b6efaf0b7b31f2f4f643381c73ccfcd28
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built pypandoc python-Levenshtein


/usr/local/lib/python3.6/dist-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')


     |████████████████████████████████| 5.5MB 7.5MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=d02265f1b2074b1bfecba7f2c51169b2dba47870965879cbb588f98881a5af4e
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_dir = '/content/gdrive/My Drive/'
base_dir = root_dir + 'Natural Language Processing/assignments/assignment_3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
def load_corpus(file_name):
    df = pd.read_csv(base_dir + file_name, index_col = 0)
    titles = df['DSI_Title'].values
    text = df['Text'].values
    return titles, text

In [ ]:
def clean_doc(doc, word2vec = False):
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stop_words = set(stopwords.words('english'))
    if word2vec == False:
        tokens = doc.split()
        tokens = lower_first_word(tokens)
        tokens = [re_punc.sub('', w) for w in tokens]
        tokens = [word for word in tokens if word.isalpha()]
        tokens = [word for word in tokens if len(word) > 5]
        # # tokens = [word.lower() for word in tokens]
        tokens = [w for w in tokens if not w in stop_words]
        # tokens = [PorterStemmer().stem(word) for word in tokens]
    elif word2vec == True:
        tokens = sent_tokenize(doc)
        tokens = [word.split() for word in tokens]
        tokens = [[re_punc.sub('', word) for word in sent] for sent in tokens]
        tokens = [[word for word in sent if word.isalpha()] for sent in tokens]
        tokens = [[word for word in sent if len(word) > 4] for sent in tokens]
        tokens = [[word.lower() for word in sent] for sent in tokens]
        tokens = [[word for word in sent if not word in stop_words] for sent in tokens]
        # tokens = [[PorterStemmer().stem(word) for word in sent] for sent in tokens]
    return tokens    

In [ ]:
def lower_first_word(tokens):
    for idx, word in enumerate(tokens):
        if '.' in word:
            if idx + 1 != len(tokens):
                tokens[idx + 1] = tokens[idx + 1].lower()
        if idx == 0:
            tokens[idx] = tokens[idx].lower()
    return tokens

In [ ]:
def process_text(file_name, method):
    titles, corpus = load_corpus(file_name)
    processed_text = []
    for text in corpus:
        if method == 'tfidf':
            text = clean_doc(text)
            temp_text = ''
            for word in text:
                temp_text += word + ' '
            processed_text.append(temp_text)
        elif method == 'word2vec':
            text = clean_doc(text, word2vec = True)
            processed_text.extend(text)
        elif method == 'doc2vec':
            text = clean_doc(text)
            processed_text.append(text)            
        elif method == 'entity_extract':            
            # processed_text.append(text.lower().split('.'))
            text = clean_doc(text, word2vec = True)
            temp_text = []
            for sent in text:
                temp_sent = ''
                for word in sent:
                    temp_sent += word + ' '
                temp_text.append(temp_sent)
            processed_text.append(temp_text)
    return titles, processed_text

In [ ]:
def df_tfidf(processed_text):
    titles, text = processed_text
    tfidf = TfidfVectorizer(ngram_range = (1,3))
    tfidf_matrix = tfidf.fit_transform(text)
    matrix = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf.get_feature_names(),
                      index = titles)
    return matrix

In [ ]:
def get_top_tfidf_terms(df_tfidf, top_num):
    top_term = {}
    for idx in range(len(df_tfidf)):
        row_max = max(df_tfidf.iloc[idx])
        for word in df_tfidf.columns:
            if df_tfidf[word].iloc[idx] == row_max:
                top_term[df_tfidf.index[idx]] = word
    return top_term

In [ ]:
def get_top_tfidf_terms(df_tfidf, top_num, value_range=False):
    top_term = {}
    for idx in range(len(df_tfidf)):
        large_values = df_tfidf.iloc[idx].nlargest(top_num)
        if value_range == False:
            term = large_values.index[top_num - 1]
            value = large_values.values[top_num - 1]
            top_term[df_tfidf.index[idx]] = term
        elif value_range == True:
            term = large_values.index[: (top_num - 1)]
            value = large_values.values[: (top_num - 1)]
            top_term[df_tfidf.index[idx]] = term
    return top_term

In [ ]:
def parse_sentences(sent):
    ent1 = ""
    ent2 = ""
    prv_tok_dep = ""
    prv_tok_text = ""
    prefix = ""
    modifier= ""
    token = ""

    remove_words = set()
    add_words = set()
    remove_from_remove_words = set()
    not_person_type = set()
    for tok in nlp(sent):
        if tok.dep_ == "advmod":
            remove_words.add(tok.text)
        if tok.dep_ == 'amod':
            remove_words.add(tok.text)
        if tok.ent_type_ == 'PERSON':
            remove_words.add(tok.text)
        else: 
            not_person_type.add(tok.text)
    return remove_words, not_person_type

In [ ]:
def get_remove_words():
    docs = process_text('class_corpus.csv', 'entity_extract')
    entities = set()
    add_entities = set()
    entities_dict = {}
    for idx, doc in enumerate(docs[1]):
        for sent in doc:
            temp_rem_list, temp_tot_list = parse_sentences(sent)
            for word in temp_rem_list:
                entities.add(word)
                if word in entities_dict.keys():
                    entities_dict[word]['pers_count'] += 1
                else:
                    entities_dict[word] = {'pers_count': 1, 'tot_count': 0}
            for word in temp_tot_list:
                if word in entities_dict.keys():
                    entities_dict[word]['tot_count'] += 1
                else:
                    entities_dict[word] = {'pers_count': 0, 'tot_count': 1}   
    entities_copy = entities.copy()
    for word in entities_copy:
        if entities_dict[word]['pers_count'] < entities_dict[word]['tot_count'] / 2:
            entities.remove(word)
    return entities

In [ ]:
remove_words = get_remove_words()

In [ ]:
class KnowledgeGraph:
    def __init__(self):
        self.graph = nx.DiGraph()

    def get_top_terms(self, num_terms, method, all_doc_term_num=None, doc_term_num=None):
        top_terms_dict = {}
        if method == 'tfidf':        
            top_terms = get_top_tfidf_terms(df_tfidf(process_text('class_corpus.csv', 
                                            'tfidf')), num_terms, value_range=True)
            for key_review, values in top_terms.items():
                nodes = combinations(values, 2)
                top_terms_dict[key_review] = list(nodes)
        elif method == 'raw':
            top_terms_dict = get_top_raw_terms(all_doc_term_num, doc_term_num)
        elif method == 'entity_extraction':
            top_terms_dict = get_top_terms_entity_extraction()
        return top_terms_dict

    def filter_top_terms(self, term, num_terms, method):
        filtered_terms = {}
        top_terms = self.get_top_terms( num_terms, method, all_doc_term_num=None, doc_term_num=None)
        for key_review, values in top_terms.items():
            for x, y in values:
                if term in x:
                    if key_review in filtered_terms.keys():
                        filtered_terms[key_review].append((x, y))
                    else:
                        filtered_terms[key_review] = [(x, y)]
                elif term in y:
                    if key_review in filtered_terms.keys():
                        filtered_terms[key_review].append((x, y))
                    else:
                        filtered_terms[key_review] = [(x, y)]
        return filtered_terms

    def set_node_attributes(self):
        edge_attributes = nx.get_edge_attributes(self.graph, 'review')
        for keys, value in edge_attributes.items():
            nx.set_node_attributes(self.graph, {keys[0]:value}, 'review')
            nx.set_node_attributes(self.graph, {keys[1]:value}, 'review')

    def get_node_class_dict(self):
        iterable = nx.get_edge_attributes(self.graph, 'review')
        node_class_dict = {}
        for keys, value in iterable.items():
            if keys[0] in node_class_dict.keys():
                if value in node_class_dict[keys[0]]:
                    pass
                else:
                    node_class_dict[keys[0]].append(value)
            else:
                node_class_dict[keys[0]] = [value]
            if keys[1] in node_class_dict.keys():
                if value in node_class_dict[keys[0]]:
                    pass
                else:
                    node_class_dict[keys[1]].append(value)
            else:
                node_class_dict[keys[1]] = [value]
        return node_class_dict

    def process_kg(self, method, num_terms=None, term=None, filtered=False,
                   all_doc_term_num=None, doc_term_num=None):
        if filtered == False:
            data = self.get_top_terms(num_terms, method, all_doc_term_num, doc_term_num)
            for key_review, nodes in data.items():
                self.graph.add_edges_from(nodes, review=key_review)
        elif filtered == True:
            data = self.filter_top_terms(term, num_terms, method)
            for key_review, nodes in data.items():
                self.graph.add_edges_from(nodes, review=key_review)
        self.set_node_attributes()

    def get_top_classes(self, num_degrees):
        top_classes = []
        for node, degree in self.graph.degree:
            if degree > num_degrees:
                top_classes.append(node)
        top_class_dict = {}
        node_class_dict = self.get_node_class_dict()
        for node in top_classes:
            top_class_dict[node] = node_class_dict.get(node)
        return top_class_dict

    def get_reviews_class(self, term):
        node_class_dict = self.get_node_class_dict()
        return node_class_dict.get(term)

    def filter_movie(self, terms):
        filtered_dict = {}
        for term in terms:
            for edges, value in nx.get_edge_attributes(self.graph, 'review').items():
                if value == term:
                    filtered_dict[edges] = value
        kg_filter = nx.DiGraph()
        kg_filter.add_edges_from(filtered_dict.keys(), review=filtered_dict.values())
        self.graph_kg(graph_data=kg_filter)
   
    def graph_kg(self, graph_data=None, attributes=False):
        if graph_data is None:
            graph_data = self.graph
        plt.figure(figsize = [12, 12])
        pos = nx.spring_layout(graph_data, k=1)
        nx.draw(graph_data, with_labels=True, node_color='skyblue', 
                edge_cmap=plt.cm.Blues, pos=pos, width=0.5,
                font_size=20, arrows=False)
        if attributes == True:
            edge_labels = nx.get_edge_attributes(graph_data, 'review')
            nx.draw_networkx_edge_labels(graph_data, pos, labels=edge_labels)

    def graph_abstract_nodes(self, num_degrees, exclude_words=None, attributes=False):
        graph = nx.DiGraph()
        for node, degree in self.graph.degree:
            if exclude_words is not None: 
                if node not in exclude_words:                
                    if degree > num_degrees:
                        graph.add_node(node)
            else:
                if degree > num_degrees:
                    graph.add_node(node)
        for edge in list(self.graph.edges):
            if (edge[0] in graph.nodes) and (edge[1] in graph.nodes):
                graph.add_edge(edge[0], edge[1])
        self.graph_kg(graph_data=graph, attributes=attributes)

In [ ]:
class BFSGraph:
    def __init__(self, kg):
        self.graph = kg
        self.visited_dict = defaultdict(list)
        self.queue = defaultdict(set)
        self.class_movie_rev_map = defaultdict(set)
        self.test_node_attributes = []

    def BFS(self, node, edge=None):
        self.queue[node].add(node)
        if edge is None:
            queue = list(self.graph.edges(node))
        else:
            queue = list(self.graph.edges(edge))
        while queue:
            s = queue.pop(0)
            for _, i in self.graph.edges(s):    
                time.sleep(0.0)            
                if i not in self.queue[node]:
                    self.test_node_attributes.append(nx.get_node_attributes(kg.graph, 'review').get(i))
                    self.queue[node].add(i)
                    temp_movie_reviews = nx.get_node_attributes(kg.graph, 'review').get(i)
                    self.class_movie_rev_map[node].add(temp_movie_reviews)
                    self.BFS(node, edge=i)

    def run(self):
        node_degrees = [len(self.graph.edges(node)) for node in self.graph.nodes]
        node_deg_std = np.std(node_degrees)
        node_deg_mean = np.mean(node_degrees)
        for node in self.graph.nodes('circle'):
            # if len(self.graph.edges(node)) > node_deg_mean + node_deg_std:
            if len(self.graph.edges(node)) > node_deg_mean:
                self.BFS(node[0])

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format(base_dir + 'GoogleNews-vectors-negative300.bin.gz', binary = True)
word_vectors_corpus = Word2Vec(process_text('class_corpus.csv', 'word2vec')[1], size=100, min_count=3, window=6)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-08-27 10:53:43,177 WARNING:gensim.models.base_any2vec:1386:      _log_train_end under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [ ]:
class WordVectors:
    def __init__(self, kg, bfsgraph, word_vectors, word_vectors_corpus, remove_words, num_terms):
        self.kg = kg
        self.bfsgraph = bfsgraph
        self.word_vectors = word_vectors
        self.word_vectors_corpus = word_vectors_corpus
        self.filtered_queue = defaultdict(set)
        self.remove_words = remove_words
        self.num_terms = num_terms

    def filter_list(self):
        for key, values in self.bfsgraph.queue.items():
            for value in values:
                if ((value in self.word_vectors.wv.vocab) and 
                    (value in self.word_vectors_corpus.wv.vocab)
                    ):
                    self.filtered_queue[key].add(value)

    def get_similarities(self):
        self.similarity_dict = {}
        self.similarity_dict_corpus = {}
        self.similarity_dict_movie = {}
        self.filter_list()
        for key, values in self.filtered_queue.items():
            self.similarity_dict[key] = defaultdict(list)
            self.similarity_dict_corpus[key] = defaultdict(list)
            self.similarity_dict_movie[key] = defaultdict(list)
            for i in permutations(values, 2):
                self.similarity_dict[key][i[0]].append(self.word_vectors.similarity(i[0], i[1]))
                self.similarity_dict_corpus[key][i[0]].append(self.word_vectors_corpus.similarity(i[0], i[1]))
                self.similarity_dict_movie[key][i[0]] = self.word_vectors_corpus.similarity(i[0], 'movie')

    def get_similarity_scores(self, weights=None):
        self.get_similarities()
        similarity_dict_results = {}
        for key, values in self.similarity_dict.items():
            # similarity_dict_results[key] = defaultdict(dict)
            similarity_dict_results[key] = {}
            for key_term in values.keys():
                wv_value = np.mean(self.similarity_dict[key][key_term])
                wv_corpus_value = np.mean(self.similarity_dict_corpus[key][key_term])
                wv_movie_value = self.similarity_dict_movie[key][key_term]
                if weights is not None:
                    similarity_dict_results[key][key_term] = weights[0] * wv_value + weights[1] * wv_corpus_value + weights[2] * wv_movie_value
                else:
                    weights = [0.5, 0.25, 0.25]
                    similarity_dict_results[key][key_term] = weights[0] * wv_value + weights[1] * wv_corpus_value + weights[2] * wv_movie_value
        return similarity_dict_results

    def get_max_similarities(self, weights=None, max_weights=None):
        similarity_scores = self.get_similarity_scores(weights)
        max_similarity_scores = defaultdict(dict)
        # max_similarity_scores = {}
        for key, values in similarity_scores.items():
            if len(values.values()) > 0:
                sorted_scores = sorted(values.items(), key=lambda x: x[1], reverse=True)[:5]
                for term in sorted_scores:
                    num_nodes = len(bfsgraph.class_movie_rev_map[term[0]])
                    kg_deg = len(self.kg.graph.edges(term[0]))
                    max_similarity_scores[key][term[0]] = {'num_nodes': num_nodes,
                                                            'sim_score': term[1],
                                                            'kg_deg': kg_deg}
        return max_similarity_scores     

    def calculate_scores(self, weights=None, max_weights=None):
        max_similarity_scores = self.get_max_similarities(weights, max_weights)
        num_nodes_max = []
        sim_score_max = []
        kg_deg_max = []
        for values in max_similarity_scores.values():
            for terms, scores in values.items():
                num_nodes_max.append(scores['num_nodes'])
                sim_score_max.append(scores['sim_score'])
                kg_deg_max.append(scores['kg_deg'])
        num_nodes_max = max(num_nodes_max)
        sim_score_max = max(sim_score_max)
        kg_deg_max = max(kg_deg_max)
        if max_weights is None:
            max_weights = {}
            max_weights['num_nodes'] = 0.25
            max_weights['sim_score'] = 0.4
            max_weights['kg_deg'] = 0.35
        for key in max_similarity_scores.keys():
            for max_val_key, max_val_val in max_similarity_scores[key].items():
                if max_val_val['num_nodes'] > 0:
                    num_nodes_score = (max_val_val['num_nodes'] / num_nodes_max) * max_weights['num_nodes']
                else:
                    num_nodes_score = 0
                sim_score_score = (max_val_val['sim_score'] / sim_score_max) * max_weights['sim_score']
                kg_deg_score = (max_val_val['kg_deg'] / kg_deg_max) * max_weights['kg_deg']
                max_similarity_scores[key][max_val_key]['agg_score'] = round((
                    num_nodes_score + sim_score_score + kg_deg_score), 3)
        return max_similarity_scores

    def filter_max_simiarity_scores(self, weights=None, max_weights=None):
        max_similarity_scores = self.calculate_scores(weights, max_weights)
        filtered_max_similarity_scores = {}
        for val in max_similarity_scores.values():
            for key, values in val.items():
                if key not in self.remove_words:
                    filtered_max_similarity_scores[key] = values['agg_score']
        return sorted(filtered_max_similarity_scores.items(), key=lambda x: x[1], reverse=True)[:5]

    def get_movie(self, weights=None, max_weights=None):
        filtered_list = self.filter_max_simiarity_scores(weights, max_weights)
        movies = {}
        filtered_list_terms = [i[0] for i in filtered_list]
        for term in filtered_list_terms:
            for bfs_term, values in bfsgraph.queue.items():
                if term in values:
                    movies[term] = bfsgraph.class_movie_rev_map[bfs_term]    
        return movies
    
    def get_class(self, weights=None, max_weights=None):
        movies = self.get_movie(weights, max_weights)
        top_terms = get_top_tfidf_terms(df_tfidf(process_text('class_corpus.csv', 
                                                'tfidf')), self.num_terms, value_range=True)
        movie_map = defaultdict(dict)
        for key, values in movies.items():
            for movie in values:
                movie_top_terms = top_terms[movie]
                sim_score = 0
                for top_term in movie_top_terms:
                    if top_term in self.word_vectors_corpus.wv.vocab:
                        sim_score += self.word_vectors_corpus.similarity(key, top_term)
                    elif top_term in self.word_vectors.wv.vocab:
                        sim_score += self.word_vectors.similarity(key, top_term)
                    else:
                        pass
                movie_map[movie][key] = sim_score

        movie_map_class = {}
        for key, values in movie_map.items():
            val_list = list(values.values())
            key_list = list(values.keys())
            index_num = val_list.index(max(val_list))
            movie_map_class[key] = key_list[index_num]

        return movie_map_class


In [ ]:
num_terms = 13
kg = KnowledgeGraph()
kg.process_kg('tfidf', num_terms=num_terms)
bfsgraph = BFSGraph(kg.graph)
bfsgraph.run()
wv_results = WordVectors(kg, bfsgraph, word_vectors, word_vectors_corpus, remove_words, num_terms)
sim_scores = wv_results.get_class(weights = [0.2, 0.1, 0.7], max_weights={'num_nodes': 0.4, 'sim_score': 0.3, 'kg_deg': 0.3})
set(sim_scores.values())
# print(len(sim_scores))
# set(sim_scores.values())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:125: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() ins

{'characters', 'effects', 'predator', 'replicants', 'terminator'}

In [ ]:
class PrepareDataNN:
    def __init__(self, classes, method, split_sent=False):
        self.classes = classes
        self.text = process_text('class_corpus.csv', method)
        self.split_sent = split_sent

    def join_data(self):
        text_dict = {self.text[0][i]: self.text[1][i] for i in range(len(self.text[0]))}        
        x_train = []
        y_train = []
        if self.split_sent == False:    
            for key in self.classes.keys():
                x_train.append(text_dict.get(key))
                y_train.append(self.classes.get(key))
        elif self.split_sent == True:
            for key in self.classes.keys():
                temp_text = text_dict.get(key)
                num_seq = 1
                seq_len = len(temp_text) // num_seq
                for i in range(num_seq):
                    x_train.append(temp_text[i*seq_len: (i+1)*seq_len])
                    y_train.append(self.classes.get(key))
        return x_train, y_train

    def prepare_data(self):
        x_train, y_train = self.join_data()
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(x_train)
        sequences = self.tokenizer.texts_to_sequences(x_train)
        y_train = LabelEncoder().fit_transform(list(y_train))
        sequence_lengths = [np.shape(i)[0] for i in sequences]
        max_len = int(max(sequence_lengths) * 0.8)
        x_train = preprocessing.sequence.pad_sequences(sequences, maxlen = max_len)
        return x_train, y_train

    def split_data(self):
        train_data, train_labels = self.prepare_data()
        train_data, val_data, train_labels, val_labels = train_test_split(
            train_data, train_labels, test_size = 0.2)
        return train_data, val_data, train_labels, val_labels

In [ ]:
class BuildModel:
    def __init__(self, sim_scores, method, split_sent=True):
        self.prepare_data = PrepareDataNN(sim_scores, method, split_sent=split_sent)
        self.x_train, self.x_val, self.y_train, self.y_val = self.prepare_data.split_data()
        keras.backend.clear_session()

    def model_structure(self):
        vocab_size = len(self.prepare_data.tokenizer.word_index) + 1
        output_size = len(set(self.y_train))
        self.model = Sequential()
        self.model.add(Embedding(vocab_size, 50, input_length=len(self.x_train[0])))
        self.model.add(LSTM(20))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(30, activation='relu'))
        self.model.add(Dense(output_size, activation='softmax'))

    def model_compile(self):
        self.model.compile(
            loss = 'sparse_categorical_crossentropy',
            optimizer = 'adam',
            metrics = ['accuracy'])

    def model_fit(self):
        self.model.fit(self.x_train, 
                       self.y_train, 
                       epochs = 40, 
                       validation_data = (self.x_val, self.y_val))

    def run(self):
        self.model_structure()
        self.model_compile()
        print(self.model.summary())
        self.model_fit()
        return self.model

In [ ]:
model = BuildModel(sim_scores, 'doc2vec').run()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 173, 50)           216700    
_________________________________________________________________
lstm (LSTM)                  (None, 20)                5680      
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                630       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 155       
Total params: 223,165
Trainable params: 223,165
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/40
2/2 [==============================] - 1s 257ms/step - loss: 1.6081 - accuracy: 0.2683 - val_loss: 1